# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

from keras import backend as K

#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
housing_df = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing_df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing_df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
x_data = housing_df.drop(['medianHouseValue'], axis=1)

In [6]:
y_data = housing_df['medianHouseValue']

In [7]:
type(x_data)

pandas.core.frame.DataFrame

In [8]:
type(y_data)

pandas.core.series.Series

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [18]:
scaler = MinMaxScaler()
scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [29]:
scaled_x_train = pd.DataFrame(data=scaler.transform(x_train), columns=x_train.columns, index=x_train.index)
scaled_x_test = pd.DataFrame(data=scaler.transform(x_test), columns=x_test.columns, index=x_test.index)

In [27]:
scaled_x_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
0,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
1,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
2,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
3,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
4,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


In [31]:
scaled_x_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [32]:
housing_df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [33]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [34]:
feat_cols = [age,rooms,bedrooms,pop,households,income]
feat_cols

[_NumericColumn(key='housingMedianAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='totalBedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='households', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='medianIncome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [35]:
input_function = tf.estimator.inputs.pandas_input_fn(scaled_x_train, y_train, batch_size=10, num_epochs=1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [38]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_task_type': 'worker', '_save_summary_steps': 100, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000110AF6803C8>, '_save_checkpoints_steps': None, '_model_dir': 'C:\\Users\\STRIKE~1\\AppData\\Local\\Temp\\tmpj6ulqaga', '_task_id': 0, '_num_ps_replicas': 0, '_session_config': None, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_service': None}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [39]:
model.train(input_fn=input_function, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\STRIKE~1\AppData\Local\Temp\tmpj6ulqaga\model.ckpt.
INFO:tensorflow:step = 1, loss = 311637280000.0
INFO:tensorflow:global_step/sec: 300.398
INFO:tensorflow:step = 101, loss = 859076100000.0 (0.334 sec)
INFO:tensorflow:global_step/sec: 348.544
INFO:tensorflow:step = 201, loss = 865530940000.0 (0.286 sec)
INFO:tensorflow:global_step/sec: 341.407
INFO:tensorflow:step = 301, loss = 789127000000.0 (0.293 sec)
INFO:tensorflow:global_step/sec: 340.245
INFO:tensorflow:step = 401, loss = 501883200000.0 (0.294 sec)
INFO:tensorflow:global_step/sec: 340.246
INFO:tensorflow:step = 501, loss = 259538600000.0 (0.295 sec)
INFO:tensorflow:global_step/sec: 352.227
INFO:tensorflow:step = 601, loss = 409774650000.0 (0.283 sec)
INFO:tensorflow:global_step/sec: 346.134
INFO:tensorflow:step = 701, loss = 173768440000.0 (0.290 sec)
INFO:tensorflow:global_step/sec: 348.545
INFO:tensorflow:step = 801, loss = 2607

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [40]:
pred_input_function = tf.estimator.inputs.pandas_input_fn(scaled_x_test, batch_size=10, num_epochs=1, shuffle=False)

In [41]:
predictions = list(model.predict(pred_input_function))

INFO:tensorflow:Restoring parameters from C:\Users\STRIKE~1\AppData\Local\Temp\tmpj6ulqaga\model.ckpt-1000


In [42]:
predictions

[{'predictions': array([81235.23], dtype=float32)},
 {'predictions': array([94088.04], dtype=float32)},
 {'predictions': array([80960.414], dtype=float32)},
 {'predictions': array([64451.11], dtype=float32)},
 {'predictions': array([85901.164], dtype=float32)},
 {'predictions': array([73628.914], dtype=float32)},
 {'predictions': array([84196.15], dtype=float32)},
 {'predictions': array([74975.24], dtype=float32)},
 {'predictions': array([74040.12], dtype=float32)},
 {'predictions': array([57845.832], dtype=float32)},
 {'predictions': array([75160.76], dtype=float32)},
 {'predictions': array([83586.01], dtype=float32)},
 {'predictions': array([70145.9], dtype=float32)},
 {'predictions': array([63756.18], dtype=float32)},
 {'predictions': array([93331.51], dtype=float32)},
 {'predictions': array([61065.883], dtype=float32)},
 {'predictions': array([74667.555], dtype=float32)},
 {'predictions': array([65612.305], dtype=float32)},
 {'predictions': array([64065.566], dtype=float32)},
 {'pr

In [46]:
preds = []
for pred in predictions:
    preds.append(pred['predictions'])

In [47]:
preds

[array([81235.23], dtype=float32),
 array([94088.04], dtype=float32),
 array([80960.414], dtype=float32),
 array([64451.11], dtype=float32),
 array([85901.164], dtype=float32),
 array([73628.914], dtype=float32),
 array([84196.15], dtype=float32),
 array([74975.24], dtype=float32),
 array([74040.12], dtype=float32),
 array([57845.832], dtype=float32),
 array([75160.76], dtype=float32),
 array([83586.01], dtype=float32),
 array([70145.9], dtype=float32),
 array([63756.18], dtype=float32),
 array([93331.51], dtype=float32),
 array([61065.883], dtype=float32),
 array([74667.555], dtype=float32),
 array([65612.305], dtype=float32),
 array([64065.566], dtype=float32),
 array([77695.77], dtype=float32),
 array([60865.965], dtype=float32),
 array([83018.41], dtype=float32),
 array([77696.555], dtype=float32),
 array([60493.816], dtype=float32),
 array([70905.2], dtype=float32),
 array([88040.43], dtype=float32),
 array([69002.555], dtype=float32),
 array([55256.48], dtype=float32),
 array([66

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [48]:
from sklearn.metrics import mean_squared_error

In [50]:
mean_squared_error(y_true=y_test, y_pred=preds)**0.5

173169.72383239598

# Great Job!